In [1]:
import os
import glob
import pandas as pd

In [2]:
x_path = '../raw_data/train_RGB_tiles_jpeg/'
x_images = os.listdir(x_path)
y_path = '../raw_data/train_mask_tiles_jpeg/'
y_masks = os.listdir(y_path)
        

In [3]:
root_path = '../raw_data/'
folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
folder_path = [f'{root_path}{folder}' for folder in folders]
folder_path

['../raw_data/train_RGB_tiles_jpeg', '../raw_data/train_mask_tiles_jpeg']

In [4]:
train_images, train_mask = [], []
train_images =[f'../raw_data/train_RGB_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[0])]
train_mask = [f'../raw_data/train_mask_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[1])]

In [5]:
train_images.sort()
train_mask.sort()

In [6]:
train_df = pd.DataFrame()
# train_df['file_path'] = train_images
train_df['image_path'] = train_images
train_df['mask_path'] = train_mask

train_df.head()

,image_path,mask_path
0,../raw_data/train_RGB_tiles_jpeg/austin10_x00_...,../raw_data/train_mask_tiles_jpeg/austin10_mas...
1,../raw_data/train_RGB_tiles_jpeg/austin10_x00_...,../raw_data/train_mask_tiles_jpeg/austin10_mas...
2,../raw_data/train_RGB_tiles_jpeg/austin10_x00_...,../raw_data/train_mask_tiles_jpeg/austin10_mas...
3,../raw_data/train_RGB_tiles_jpeg/austin10_x00_...,../raw_data/train_mask_tiles_jpeg/austin10_mas...
4,../raw_data/train_RGB_tiles_jpeg/austin10_x00_...,../raw_data/train_mask_tiles_jpeg/austin10_mas...


In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def holdout(df, train_ratio=0.7, test_to_val_ratio=(0.05/0.3), include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
         (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

2022-06-09 22:30:54.404642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 22:30:54.404660: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
ds_train, ds_val, ds_test = holdout(train_df)

2022-06-09 22:30:56.942710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-09 22:30:56.942744: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-09 22:30:56.942772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hsth-ThinkPad-2nd): /proc/driver/nvidia/version does not exist
2022-06-09 22:30:56.944023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def process_path(input_path, mask_path):
    """
    Load images from files.
    :input_path: the path to the satellite file
    :mask_path: the path to the mask file
    :return: The image and mask
    .. note:: Works with jpg images 
              Only the first channel is kept for the mask
    """
    
    IMAGE_SQ_SIZE = 224

    input_img = tf.io.read_file(input_path)   
    input_img = tf.io.decode_jpeg(input_img, channels=3)
    input_img =  tf.image.resize(input_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

    mask_img = tf.io.read_file(mask_path)   
    mask_img = tf.io.decode_jpeg(mask_img, channels=1)
    mask_img =  tf.image.resize(mask_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

   
    return input_img, mask_img

In [10]:
def normalize(image, mask):
    # image = tf.cast(image, tf.float32) / 255.

    return tf.math.divide(image, 255), tf.math.divide(mask, 255) 

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(process_path) \
.map(normalize) \
.batch(batch_size=60) \
.prefetch(buffer_size=AUTOTUNE)

In [12]:
from tensorflow import keras
import tensorflow as tf

# model_path_and_filename = '../models/first_UNET_input_shape_224x224x3.h5'
# model = keras.models.load_model(model_path_and_filename)

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import VGG16

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, IoU



class SMR_Model():
    ''' creating our first lwb_smr models '''

    def __init__(self, input_shape):
        self.input_shape = input_shape

    def get_latest_model(self):
        model = self.build_vgg16_unet(self.input_shape)
        model = self.compile_model(model)

        return model

    def convolution_block(self, inputs, num_filters):
        ''' simple UNET convolution block with BatchNormalisation '''

        # convolution layer 1 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # convolution layer 2 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # max pooling not used here as just the bridge

        return x

    def decoder_block(self, inputs, skip_tensor, num_filters):
        ''' decoder block for UNET '''
        # adds in the skips with concatenate
        x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
        x = Concatenate()([x, skip_tensor])     # bringing in skip layer
        x = self.convolution_block(x, num_filters)

        return x

    def build_vgg16_unet(self, input_shape):
        ''' build vgg-16 '''

        inputs = Input(input_shape)

        # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
        vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
        # vgg16.summary()
        vgg16.trainable = False

        ''' Encoder - skip layers '''
        skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
        skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
        skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
        skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
        # display('skip4: ' + str(skip4.shape))

        # only need to specify the skip layers, as VGG16 is an Encoder
        # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

        ''' Bridge '''
        bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
        # display('bridge: ' + str(bridge.shape))


        ''' Decoder '''
        d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
        d2 = self.decoder_block(d1, skip3, 256) #  256 filters
        d3 = self.decoder_block(d2, skip2, 128) #  128 filters
        d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

        ''' Output '''
        outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

        model = Model(inputs, outputs, name='first_VGG16_UNET')

        return model

    def compile_model(self, m):
        ''' compile as a basic unet for now... first actual run '''
        m.compile(
            loss='binary_crossentropy',
            optimizer='adam'
        )
        return m

In [13]:
UNET_INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 8
EPOCHS = 2

In [14]:
# RECOMMENDATIONS FOR OVERNIGHT RUN:

# epochs = 5 (local) 

In [16]:
# x = ??? locally
# hours = (EPOCHS*x)/60
# hours

In [17]:
getVGG16 = SMR_Model(UNET_INPUT_SHAPE)
model = getVGG16.build_vgg16_unet(UNET_INPUT_SHAPE)

In [18]:
# binaryIoU metric
threshold = 0.5
binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
AUC = tf.keras.metrics.AUC()

# Compile Model
model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy', binaryIoU, AUC]
        )

model_path_and_filename = '../models/220609_v6_UNET_input_shape_224x224x3.h5'
model.save(model_path_and_filename)

In [ ]:
mc = ModelCheckpoint('../checkpoints/oxford_segmentation.h5', save_best_only=True) # could put path here
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(
    ds_train,
    validation_data=ds_val,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[mc, es]
    )

model_path_and_filename = '../models/220609_5epoch_LOCAL_UNET_input_shape_224x224x3.h5'
model.save(model_path_and_filename)


Epoch 1/2


2022-06-09 22:31:19.797895: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 770703360 exceeds 10% of free system memory.
2022-06-09 22:31:20.228455: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 770703360 exceeds 10% of free system memory.
2022-06-09 22:31:21.934097: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 385351680 exceeds 10% of free system memory.
2022-06-09 22:31:22.737430: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 385351680 exceeds 10% of free system memory.
2022-06-09 22:31:37.145165: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 385351680 exceeds 10% of free system memory.
